In [13]:
from search_engine_versions.bm25_body_title import SearchEngine as SearchEngineBM25BT
from search_engine_versions.bm25_body import SearchEngine as SearchEngineBM25B
from search_engine_versions.bm25_title import SearchEngine as SearchEngineBM25T
from search_engine_versions.hybrid import SearchEngine as SearchEngineHybrid
from search_engine_versions.max_sim_complete_backup import SearchEngine as SearchEngineMaxSim
from search_engine_versions.max_sim_global_chunks import SearchEngine as SearchEngineMaxSimGlobalChunks
from search_engine_versions.max_sim_global_sentences import SearchEngine as SearchEngineMaxSimGlobalSentences
from search_engine_versions.max_sim_local_chunks import SearchEngine as SearchEngineMaxSimLocalChunks
from search_engine_versions.max_sim_local_sentences import SearchEngine as SearchEngineMaxSimLocalSentences
from query_evaluator import evaluate_articles_boolean, evaluate_articles_levels
from query_generator import generate_and_save_queries
import numpy as np
import pandas as pd
import json


# For queries gen
# path = './app/data/example.json'
# with open(path, 'r', encoding='utf-8') as f:
#     data = f.readlines()
#     data = [json.loads(line) for line in data]
# generate_and_save_queries(data, num_queries=100, output_path='./app/data/queries.json')

DATA_DIR = "./app/data/"
DATA_FILES = ["example.json"]
SPLIT_SIZE_LIMIT = 100

search_engines = {}
search_engines_dict = {
    "bm25bt": SearchEngineBM25BT, 
    "bm25b": SearchEngineBM25B,
    "bm25t": SearchEngineBM25T,
    "hybrid": SearchEngineHybrid,
    "maxsim": SearchEngineMaxSim,
    "maxsimglobalchunks": SearchEngineMaxSimGlobalChunks,
    "maxsimglobalsentences": SearchEngineMaxSimGlobalSentences,
    "maxsimlocalchunks": SearchEngineMaxSimLocalChunks,
    "maxsimlocalsentences": SearchEngineMaxSimLocalSentences,
}

# for search_engine in search_engines_list:
#     search_engine.feed_json(DATA_DIR, DATA_FILES, SPLIT_SIZE_LIMIT)
#     search_engines[search_engine.name] = search_engine


In [14]:
def average_precision(relevances):
    relevant = np.where(relevances == 1)[0] + 1  # get positions of relevant docs, +1 because index is zero-based
    if len(relevant) == 0:
        return 0
    precision_at_k = np.cumsum(relevances) / (np.arange(len(relevances)) + 1)
    return np.sum(precision_at_k[relevances == 1]) / len(relevant)

def dcg(relevances):
    k = len(relevances)
    relevances = np.asfarray(relevances)[:k]
    if relevances.size:
        return relevances[0] + np.sum(relevances[1:] / np.log2(np.arange(2, relevances.size + 1)))
    return 0

def ndcg(relevances):
    k = len(relevances)
    dcg_val = dcg(relevances)
    ideal_relevances = sorted(relevances, reverse=True)
    idcg_val = dcg(ideal_relevances)
    return dcg_val / idcg_val if idcg_val > 0 else 0

with open(DATA_DIR + "queries.json") as f:
    queries = f.readlines()
    queries = [json.loads(line) for line in queries]


def test_search_engine(SearchEngine, name):
    search_engine = SearchEngine()
    search_engine.feed_json(DATA_DIR, DATA_FILES, SPLIT_SIZE_LIMIT)
    search_engines[name] = search_engine

    aps = {}
    ndcgs_levels = {}
    ndcgs_boolean = {}

    for query_obj in queries[:10]:
        query = query_obj['query']
        search_results = search_engine.search(query, 5)
        search_results.rename(columns={'body': 'abstract'}, inplace=True)

        evaluated_results_boolean = evaluate_articles_boolean(search_results.to_dict(orient="records"), query, verbose=False)
        evaluated_results_boolean = pd.DataFrame(evaluated_results_boolean)
        evaluated_results_boolean['eval'] = pd.to_numeric(evaluated_results_boolean['eval'])
        print(evaluated_results_boolean)

        evaluated_results_levels = evaluate_articles_levels(search_results.to_dict(orient="records"), query, verbose=False)
        evaluated_results_levels = pd.DataFrame(evaluated_results_levels)
        evaluated_results_levels['eval'] = pd.to_numeric(evaluated_results_levels['eval'])
        print(evaluated_results_levels)

        aps[query_obj['id']] = average_precision(evaluated_results_boolean['eval'])
        ndcgs_levels[query_obj['id']] = ndcg(evaluated_results_levels['eval'])
        ndcgs_boolean[query_obj['id']] = ndcg(evaluated_results_boolean['eval'])
    
    metrics = {
        'aps': aps,
        'ndcgs_levels': ndcgs_levels,
        'ndcgs_boolean': ndcgs_boolean
    }
    return metrics


In [16]:
metrics_dict = {}
idx = 0
for name, search_engine in search_engines_dict.items():
    if idx > 3:
        break
    print('running for ', name)
    metrics = test_search_engine(search_engine, name)
    metrics_dict[name] = metrics
    idx+=1
    !docker stop $(docker ps -a -q) 

running for  bm25bt
Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 10/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 15/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Application is up!
Finished deployment.


Evaluating articles:  20%|██        | 1/5 [00:01<00:07,  1.99s/it]

id: 0704.0001


Evaluating articles:  40%|████      | 2/5 [00:02<00:03,  1.12s/it]

id: 0704.0060


...

abdc641f54d4
c870ed76a2df
454c00247a31
4b414421b8a8
458574697be5
421df1fdaee9
a28a62472faf
91cf3800e4ba
cc19e3046262
bce785b59fb3
running for  bm25b
Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 10/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 15/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 20/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 25/300 seconds...
Using p

Evaluating articles:   0%|          | 0/5 [00:00<?, ?it/s]

id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.0001
Rate limit achieved: 'title'
id: 0704.00

Evaluating articles:   0%|          | 0/5 [23:34<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
from sheets_connection import SaveMetrics

save_metrics = SaveMetrics()

save_metrics.append_metrics('ndcg_binary_map', df.T)